In [3]:
#import packages
import pandas as pd
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#modify import path
import sys
sys.path.append('../our_analysis')

#import our code
from hybrid_mle import fit_hybrid_mixed_model_drifting


In [5]:
#Set WD
notebook_dir = os.getcwd()

In [ ]:
#load dataset
df = pd.read_csv("../dataset/all_subjects.csv") #our dataset

#Let's remove all of the slow trials for the df
df = df[df['slow']==0]
# We need to cast the choice data to an int
df['choice1'] = df['choice1'].astype(int)
df['choice2'] = df['choice2'].astype(int)
df['final_state'] = df['final_state'].astype(int)


story_trials = df[df["condition"] == "story"]
abstract_trials=df[df["condition"] == "abstract"]

In [ ]:
story_results_df, params, logli_story_basic = fit_hybrid_mixed_model_drifting(
    data_df=story_trials,
    stan_file="../our_analysis/driftingWeight_mle.stan",
    output_file="story_params_drift.csv"
)

abstract_results_df, params, logli_abstract_basic = fit_hybrid_mixed_model_drifting(
    data_df=abstract_trials,
    stan_file="../our_analysis/driftingWeight_mle.stan",
    output_file="abstract_params_drift.csv"
)

In [6]:
#store the results in a clean table we can use for the assignment
results =[] 

for df in [story_results_df, abstract_results_df]:
    params=df.iloc[0].to_dict()
    results.append(params)

results[0]['logli']=logli_story_basic
results[1]['logli']=logli_abstract_basic

results_df = pd.DataFrame(results)
results_df.head()
## we should probably just save this as a csv here. It has the participant field which we can drop in later formatting.
results_df.to_csv("hybrid_model_params_driftingW.csv", index=False)


In [ ]:
import pandas as pd
formatting_df = pd.read_csv("/Users/nokada/Documents/CALTECH/sdn_hw2/part3/hybrid_model_params_driftingW.csv")

num_params = 8
story_sample_size = 46
abstract_sample_size = 48

AIC_story = 2*num_params-2*formatting_df['logli'][0]
AIC_abstract = 2*num_params-2*formatting_df['logli'][1]

BIC_story = 2*num_params * np.log(story_sample_size) - 2 * formatting_df['logli'][0]
BIC_abstract = 2*num_params * np.log(abstract_sample_size) - 2 * formatting_df['logli'][1]



formatting_df['AIC'] = AIC_story
formatting_df['AIC'][1] = AIC_abstract

formatting_df['BIC'] = BIC_story
formatting_df['BIC'][1] = BIC_abstract

In [18]:
formatting_df = formatting_df.round(3)
formatting_df.to_csv("../csvs_formatted/hybrid_model_params_driftingW.csv", index=False)


In [19]:
formatting_df.head()

,participant,condition,alpha1,alpha2,lmbd,beta1,beta2,p,w0,kappa,logli,AIC,BIC
0,1,story,0.633,0.804,1.000,3.406,2.108,0.142,0.708,0.051,-7790.76,15597.52,15642.778
1,5,abstract,0.161,0.497,0.501,5.706,3.008,0.162,0.963,0.000,-7168.28,14352.56,14398.499
